Open this notebook in Google Colab: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/santoshborse/pydatanyc2024/blob/main/data-prep.ipynb)

### 1 - Setup


In [2]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   RUNNING_IN_COLAB = True
else:
   print("NOT in Colab")
   RUNNING_IN_COLAB = False
!wget -O 'my_utils.py' 'https://raw.githubusercontent.com/sujee/data-prep-kit/intro-example1/examples/notebooks/intro/my_utils.py'

Running in Colab
--2024-10-30 20:15:01--  https://raw.githubusercontent.com/sujee/data-prep-kit/intro-example1/examples/notebooks/intro/my_utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1856 (1.8K) [text/plain]
Saving to: ‘my_utils.py’

my_utils.py         100%[===================>]   1.81K  --.-KB/s    in 0s      

2024-10-30 20:15:01 (22.8 MB/s) - ‘my_utils.py’ saved [1856/1856]



### 2. Install required Packages

In [13]:
!pip install ipykernel datasets humanfriendly
!pip3 install 'data-prep-toolkit[ray]==0.2.2.dev1'

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.2/275.2 kB 15.2 MB/s eta 0:00:00
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


###

In [3]:
from huggingface_hub import hf_hub_download
!mkdir -p 'yelp-dataset'
hf_hub_download(repo_type="dataset", repo_id="yelp/yelp_review_full", filename="yelp_review_full/test-00000-of-00001.parquet", local_dir="./yelp-dataset")
hf_hub_download(repo_type="dataset", repo_id="yelp/yelp_review_full", filename="yelp_review_full/train-00000-of-00001.parquet", local_dir="./yelp-dataset")
input_folder = "yelp-dataset/yelp_review_full"


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### 3 - Inspect input


In [12]:
from my_utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(input_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

Output dimensions (rows x columns)=  (700000, 2)


,label,text
0,0,I got 'new' tires from them and within two wee...
1,0,Don't waste your time. We had two different p...
2,0,All I can say is the worst! We were the only 2...
3,0,I have been to this restaurant twice and was d...
4,0,Food was NOT GOOD at all! My husband & I ate h...


### 4. Resize

In [ ]:
!pip install 'data-prep-toolkit-transforms[resize]==0.2.2.dev1'

In [7]:
STAGE = "1"
output_folder =  "yelp-dataset/s1-resized"
from data_processing.utils import ParamsUtils


In [4]:


import os
import sys

from data_processing.runtime.pure_python import PythonTransformLauncher
from resize_transform_python import ResizePythonTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    # resize configuration
    # "resize_max_mbytes_per_table":  0.02,
    "resize_max_rows_per_table": 35000,
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = PythonTransformLauncher(ResizePythonTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")

20:34:44 INFO - Split file parameters are : {'max_rows_per_table': 35000, 'max_mbytes_per_table': -1, 'size_type': 'disk'}
INFO:resize_transform:Split file parameters are : {'max_rows_per_table': 35000, 'max_mbytes_per_table': -1, 'size_type': 'disk'}
20:34:44 INFO - pipeline id pipeline_id
INFO:data_processing.runtime.execution_configuration:pipeline id pipeline_id
20:34:44 INFO - code location None
INFO:data_processing.runtime.execution_configuration:code location None
20:34:44 INFO - data factory data_ is using local data access: input_folder - yelp-dataset/yelp_review_full output_folder - yelp-dataset/s1-resized
INFO:data_processing.data_access.data_access_factory_base4930187d-0650-4796-9eb6-ccd2ecf870a6:data factory data_ is using local data access: input_folder - yelp-dataset/yelp_review_full output_folder - yelp-dataset/s1-resized
20:34:44 INFO - data factory data_ max_files -1, n_sample -1
INFO:data_processing.data_access.data_access_factory_base4930187d-0650-4796-9eb6-ccd2ecf8

✅ Stage:1 completed successfully


In [5]:
STAGE = "2"
input_folder = output_folder
output_folder =  "yelp-dataset/s2-hap"

In [3]:
!pip install 'data-prep-toolkit-transforms[ray,hap]==0.2.2.dev1'

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
import sys, os
from data_processing.runtime.pure_python import PythonTransformLauncher
from hap_transform_python import HAPPythonTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
params = {
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # hap_params
    "model_name_or_path": 'ibm-granite/granite-guardian-hap-38m',
    "annotation_column": "hap_score",
    "doc_text_column": "text",
    "inference_engine": "CPU",
    "max_length": 512,
    "batch_size": 128,
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = PythonTransformLauncher(HAPPythonTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Job failed")

01:42:17 INFO - hap params are {'model_name_or_path': 'ibm-granite/granite-guardian-hap-38m', 'annotation_column': 'hap_score', 'doc_text_column': 'text', 'inference_engine': 'CPU', 'max_length': 512, 'batch_size': 128} 
INFO:hap_transform:hap params are {'model_name_or_path': 'ibm-granite/granite-guardian-hap-38m', 'annotation_column': 'hap_score', 'doc_text_column': 'text', 'inference_engine': 'CPU', 'max_length': 512, 'batch_size': 128} 
01:42:17 INFO - pipeline id pipeline_id
INFO:data_processing.runtime.execution_configuration:pipeline id pipeline_id
01:42:17 INFO - code location None
INFO:data_processing.runtime.execution_configuration:code location None
01:42:17 INFO - data factory data_ is using local data access: input_folder - yelp-dataset/s1-resized output_folder - yelp-dataset/s1-resized
INFO:data_processing.data_access.data_access_factory_baseea9a25ae-7c77-428d-b7b3-aee2631fe0f4:data factory data_ is using local data access: input_folder - yelp-dataset/s1-resized output_fo

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/154M [00:00<?, ?B/s]

Processing batch: 0/2317
Processing batch: 1/2317
Processing batch: 2/2317
Processing batch: 3/2317
Processing batch: 4/2317
Processing batch: 5/2317
Processing batch: 6/2317
Processing batch: 7/2317
Processing batch: 8/2317
Processing batch: 9/2317
Processing batch: 10/2317
Processing batch: 11/2317
Processing batch: 12/2317
Processing batch: 13/2317
Processing batch: 14/2317
Processing batch: 15/2317
Processing batch: 16/2317
Processing batch: 17/2317
Processing batch: 18/2317
Processing batch: 19/2317
Processing batch: 20/2317
Processing batch: 21/2317
Processing batch: 22/2317
